In [14]:
import pandas as pd
from google.cloud import storage
import os

pd.set_option('max_column',100)
pd.set_option('max_row',100)

In [68]:
def upload_statement_files_and_logs(bucket_name, data_path,ticker):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    
    files_uploaded = []
    
    timeseries_path = f'{data_path}timeseries/{ticker}/'
    log_path = f'{data_path}logs/ticker/'
    
    
    for dirname, _, filenames in os.walk(timeseries_path):
        for filename in filenames:

            source_file_name = f'{dirname}/{filename}'
            destination_blob_name = f'{dirname[len(data_path)+11:]}/{filename}'
        
    
            blob = bucket.blob(destination_blob_name)

            blob.upload_from_filename(source_file_name)

            files_uploaded.append(filename)

    for dirname, _, filenames in os.walk(log_path):
        for filename in filenames:

            source_file_name = f'{dirname}/{filename}'
            destination_blob_name = f'{dirname[len(data_path)+5:]}/{filename}'
            
            blob = bucket.blob(destination_blob_name)
            blob.upload_from_filename(source_file_name)

            files_uploaded.append(filename)

            
    print(f"Files uploaded: {', '.join(files_uploaded)}")

In [75]:
def already_uploaded(bucket_name, ticker):

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    key_files = ['Income Statement.csv','Balance Sheet.csv','Cash Flow.csv']
    key_folder = f'{ticker}/Clean Statements/'

    key_files_exist = True
    for file in key_files:    
        stats = storage.Blob(bucket=bucket, name=f'{key_folder}{file}').exists(storage_client)

        if stats == False:

            key_files_exist = False
            break

    return key_files_exist


In [78]:
data_path = '/Users/goldbloom/Dropbox/Side Projects/Edgar/data/'
ticker = 'AAPL'
bucket_name = 'kaggle_sec_data'

upload_statement_files_and_logs(bucket_name,data_path,ticker)

In [77]:
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.get_blob(ticker)

#for blob in storage_client.list_blobs(bucket_name):
#    print(blob.name)
    
file_to_check = f'{ticker}/Canonical Statement/Income Statement.csv'
    
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
stats = storage.Blob(bucket=bucket, name=file_to_check).exists(storage_client)
print(stats)

False


,,,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31,2015-09-30,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,2013-03-31,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31,2008-09-30,2008-06-30,2007-12-31
filing_label,xbrl_tag,standard_label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Revenue from Contract with Customer, Excluding Assessed Tax",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,Revenue,8.891200e+10,7.545200e+10,8.743700e+10,6.998100e+10,6.340400e+10,5.970000e+10,7.238300e+10,5.657600e+10,5.288600e+10,5.104200e+10,6.045300e+10,4.374400e+10,3.795500e+10,3.571400e+10,4.374100e+10,3.271400e+10,3.040400e+10,2.912800e+10,3.574700e+10,2.535800e+10,2.318500e+10,2.271700e+10,2.932800e+10,2.057900e+10,1.934000e+10,1.974100e+10,2.558700e+10,1.709200e+10,1.570400e+10,1.607000e+10,1.743100e+10,1.380600e+10,1.283400e+10,1.318500e+10,1.743100e+10,1.087600e+10,9.913000e+09,9.857000e+09,1.294800e+10,7.560000e+09,6.566000e+09,7.131000e+09,9.520000e+09,5.449000e+09,4.651000e+09,4.889000e+09,1.916600e+10,4.264000e+09,4.063000e+09,1.483500e+10
"Revenue from Contract with Customer, Excluding Assessed Tax (Product)",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,Revenue (Product),5.024400e+10,4.184100e+10,5.054300e+10,3.972600e+10,3.585600e+10,3.428300e+10,4.470000e+10,3.374600e+10,3.186400e+10,3.160500e+10,4.132600e+10,2.876800e+10,2.474500e+10,2.373400e+10,3.062900e+10,2.233900e+10,2.111600e+10,2.058100e+10,2.661700e+10,1.846300e+10,1.710400e+10,1.708400e+10,2.310200e+10,1.602200e+10,1.525100e+10,1.570500e+10,2.107200e+10,1.380800e+10,1.275200e+10,1.327100e+10,1.814700e+10,1.154600e+10,1.079100e+10,1.124900e+10,NaN,NaN,NaN,NaN,3.079200e+10,NaN,NaN,NaN,2.227300e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax (Service)",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,Revenue (Service),3.866800e+10,3.361100e+10,3.689400e+10,3.025500e+10,2.754800e+10,2.541700e+10,2.768300e+10,2.283000e+10,2.102200e+10,1.943700e+10,1.912700e+10,1.497600e+10,1.321000e+10,1.198000e+10,1.311200e+10,1.037500e+10,9.288000e+09,8.547000e+09,9.129000e+09,6.895000e+09,6.081000e+09,5.633000e+09,6.226000e+09,4.557000e+09,4.089000e+09,4.036000e+09,4.514000e+09,3.284000e+09,2.952000e+09,2.799000e+09,3.121000e+09,2.260000e+09,2.043000e+09,1.936000e+09,NaN,NaN,NaN,NaN,3.412000e+09,NaN,NaN,NaN,2.236000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Goods and Services Sold,us-gaap_costofgoodsandservicessold,Cost of revenue,5.266000e+10,4.425700e+10,5.397700e+10,4.130200e+10,3.633700e+10,3.392000e+10,4.478600e+10,3.300300e+10,3.063200e+10,3.073500e+10,3.849400e+10,2.754900e+10,2.345100e+10,2.244000e+10,2.895900e+10,2.126000e+10,1.918000e+10,1.886600e+10,2.434100e+10,1.675500e+10,1.516000e+10,1.539500e+10,2.067100e+10,1.462700e+10,1.339900e+10,1.405500e+10,1.880500e+10,1.236600e+10,1.120900e+10,1.180100e+10,1.613700e+10,1.031900e+10,9.488000e+09,1.002700e+10,1.383000e+10,8.325000e+09,7.525000e+09,7.608000e+09,1.031700e+10,5.786000e+09,4.957000e+09,5.501000e+09,7.543000e+09,4.176000e+09,3.518000e+09,3.741000e+09,1.489600e+10,3.265000e+09,3.096000e+09,1.148200e+10
Fulfillment Expense,amzn_fulfillmentexpense,NaN,1.380600e+10,1.153100e+10,1.219300e+10,1.016700e+10,9.271000e+09,8.601000e+09,1.002800e+10,8.275000e+09,7.932000e+09,7.792000e+09,8.974000e+09,6.420000e+09,5.158000e+09,4.697000e+09,5.719000e+09,4.335000e+09,3.878000e+09,3.687000e+09,4.545000e+09,3.230000e+09,2.876000e+09,2.759000e+09,3.424000e+09,2.643000e+09,2.382000e+09,2.317000e+09,2.918000e+09,2.034000e+09,1.837000e+09,1.796000e+09,2.258000e+09,

In [175]:
ticker = 'AMZN'
df = pd.read_csv(f'{data_path}timeseries/{ticker}/Clean Statements/Income Statement.csv',index_col=[0,1,2])

#df_gross_income = pd.DataFrame(df[df.index.get_level_values(2) == 'Revenue'].values - df[df.index.get_level_values(2) == 'Cost of revenue'].values,index=pd.MultiIndex.from_tuples(list(zip([None],[None],['Gross income'])),names=['filing_label','xbrl_tag','standard_label']),columns=df.columns)
#split_df_loc = df.index.get_level_values(2).get_loc('Cost of revenue')+1

#df_new = pd.concat([df.iloc[:split_df_loc], df_gross_income, df.iloc[split_df_loc:]])

In [176]:
df

,,,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31,2015-09-30,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,2013-03-31,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31,2008-09-30,2008-06-30,2007-12-31
filing_label,xbrl_tag,standard_label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Revenue from Contract with Customer, Excluding Assessed Tax",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,Revenue,8.891200e+10,7.545200e+10,8.743700e+10,6.998100e+10,6.340400e+10,5.970000e+10,7.238300e+10,5.657600e+10,5.288600e+10,5.104200e+10,6.045300e+10,4.374400e+10,3.795500e+10,3.571400e+10,4.374100e+10,3.271400e+10,3.040400e+10,2.912800e+10,3.574700e+10,2.535800e+10,2.318500e+10,2.271700e+10,2.932800e+10,2.057900e+10,1.934000e+10,1.974100e+10,2.558700e+10,1.709200e+10,1.570400e+10,1.607000e+10,1.743100e+10,1.380600e+10,1.283400e+10,1.318500e+10,1.743100e+10,1.087600e+10,9.913000e+09,9.857000e+09,1.294800e+10,7.560000e+09,6.566000e+09,7.131000e+09,9.520000e+09,5.449000e+09,4.651000e+09,4.889000e+09,1.916600e+10,4.264000e+09,4.063000e+09,1.483500e+10
"Revenue from Contract with Customer, Excluding Assessed Tax (Product)",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,Revenue (Product),5.024400e+10,4.184100e+10,5.054300e+10,3.972600e+10,3.585600e+10,3.428300e+10,4.470000e+10,3.374600e+10,3.186400e+10,3.160500e+10,4.132600e+10,2.876800e+10,2.474500e+10,2.373400e+10,3.062900e+10,2.233900e+10,2.111600e+10,2.058100e+10,2.661700e+10,1.846300e+10,1.710400e+10,1.708400e+10,2.310200e+10,1.602200e+10,1.525100e+10,1.570500e+10,2.107200e+10,1.380800e+10,1.275200e+10,1.327100e+10,1.814700e+10,1.154600e+10,1.079100e+10,1.124900e+10,NaN,NaN,NaN,NaN,3.079200e+10,NaN,NaN,NaN,2.227300e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax (Service)",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,Revenue (Service),3.866800e+10,3.361100e+10,3.689400e+10,3.025500e+10,2.754800e+10,2.541700e+10,2.768300e+10,2.283000e+10,2.102200e+10,1.943700e+10,1.912700e+10,1.497600e+10,1.321000e+10,1.198000e+10,1.311200e+10,1.037500e+10,9.288000e+09,8.547000e+09,9.129000e+09,6.895000e+09,6.081000e+09,5.633000e+09,6.226000e+09,4.557000e+09,4.089000e+09,4.036000e+09,4.514000e+09,3.284000e+09,2.952000e+09,2.799000e+09,3.121000e+09,2.260000e+09,2.043000e+09,1.936000e+09,NaN,NaN,NaN,NaN,3.412000e+09,NaN,NaN,NaN,2.236000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Goods and Services Sold,us-gaap_costofgoodsandservicessold,Cost of revenue,5.266000e+10,4.425700e+10,5.397700e+10,4.130200e+10,3.633700e+10,3.392000e+10,4.478600e+10,3.300300e+10,3.063200e+10,3.073500e+10,3.849400e+10,2.754900e+10,2.345100e+10,2.244000e+10,2.895900e+10,2.126000e+10,1.918000e+10,1.886600e+10,2.434100e+10,1.675500e+10,1.516000e+10,1.539500e+10,2.067100e+10,1.462700e+10,1.339900e+10,1.405500e+10,1.880500e+10,1.236600e+10,1.120900e+10,1.180100e+10,1.613700e+10,1.031900e+10,9.488000e+09,1.002700e+10,1.383000e+10,8.325000e+09,7.525000e+09,7.608000e+09,1.031700e+10,5.786000e+09,4.957000e+09,5.501000e+09,7.543000e+09,4.176000e+09,3.518000e+09,3.741000e+09,1.489600e+10,3.265000e+09,3.096000e+09,1.148200e+10
NaN,NaN,Gross income,3.625200e+10,3.119500e+10,3.346000e+10,2.867900e+10,2.706700e+10,2.578000e+10,2.759700e+10,2.357300e+10,2.225400e+10,2.030700e+10,2.195900e+10,1.619500e+10,1.450400e+10,1.327400e+10,1.478200e+10,1.145400e+10,1.122400e+10,1.026200e+10,1.140600e+10,8.603000e+09,8.025000e+09,7.322000e+09,8.657000e+09,5.952000e+09,5.941000e+09,5.686000e+09,6.782000e+09,4.726000e+09,4.495000e+09,4.269000e+09,1.294000e+09,3.487000e+09,3.346000e+09,3

In [112]:
8.891200e+10-5.266000e+10

36252000000.0

[(None, None, 'Gross income')]

MultiIndex([(nan, nan, 'Gross income')],
           names=['filing_label', 'xbrl_tag', 'standard_label'])